In [1]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime, inverse, GCD
import random

# Prerequisites

- Coppersmith
- GCD
- RSA

# Theory

**Task**
- The attack exploits the fact that two messages might be related by some polynomial

**Theorem Franklin-Reiter**  
Set $e = 3$ and let $⟨ N , e ⟩$ be an RSA public key. Let $M_1 ≠ M_2 ∈ \mathbb{Z}^*$ satisfy $ M_{1}\equiv f(M_{2}) \bmod {N}$ for some linear polynomial $f = a x + b ∈ \mathbb{Z}_N[ x ]$ with $b ≠ 0$. Then, given $⟨ N , e , C_1 , C_2 , f ⟩$, an attacker Eve, can recover $M_1 , M_2$ in time quadratic in $\log_2 (N)$.

**Proof**
- $M_1 \equiv a*M_2 + b \bmod N$
- $C_1 \equiv M_1^e \equiv (a*M_2 + b)^e \bmod N <=> 0 \equiv (a*M_2 + b)^e - C_1 \bmod N $
- $C_2 \equiv (M_2)^e \bmod N <=> 0 \equiv M_2^e - C_2 \bmod N $
- If we want to solve for $M_2$ construct the polynomials
    - $g_1(x) = (a*x + b) ^e - C_1 \in Z_N[x]$
    - $g_2(x) = x^e - C_2 \in Z_N[x]$
- $M_2$ is a root to both polynomial => $x - M_2$ divides both polynomials => we can compute the GCD to both polynomials and if it's linear we found $M_2$


# Code

In [2]:
# Construct rsa keys
def get_keys():
    p = getPrime(1024)
    q = getPrime(1024)
    N = p * q
    e = 3
    assert GCD(N, e) == 1
    phi = (p-1)*(q-1)
    d = inverse(e, phi)
    return (N, e), (p, q, d)

In [3]:
#get keys
(N, e), (p, q, d) = get_keys()

In [4]:
#Construct polynomial
R.<x> = PolynomialRing(Zmod(N))
a = random.randint(2, N-2)
b = random.randint(2, N-1)
f = a*x + b


In [5]:
def poly_gcd(f1, f2):
    while f2:
        f1, f2, = f2, f1 % f2
    return f1.monic()
def franklin_reiter(f1, f2, c1, c2, e, N):
    #R = f1.parent()
    g1 = f1**e - c1
    g2 = f2**e - c2
    g = poly_gcd(g1, g2)
    print(g)
    return -g.coefficients()[0]

In [6]:
m2 = int(bytes_to_long(b'this_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_message')) 
m1 = int(f(m2)) #overwrite Integer class in sage

In [7]:
c2 = pow(m2, e, N)
c1 = pow(m1, e, N)

In [11]:
f2 = x
m2_decr = franklin_reiter(f, f2, c1, c2, e, N)
m2_decr = int(m2_decr)

x + 21203948306901267409690734889935566282659108778871678850540421071839030494727929669801476386512031853123849490159693445320002025068724545740890762962520393785396760795180105117560107985843716188291578481001836170553565564043720040842980538654474358765009340657862068466488908878538881547302021627565407737248266298492693382271432885042151752160052320890882999893108464403020979808735925803880704617073718450274977513402295476480271240345863198735818606252926319213543623487874482689527748200225173490003401561280539995256927759991161487361413669822488123898287181545677977749483333753357897709470657408575254391642272


In [12]:
long_to_bytes(m2_decr)

b'this_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_message'

In [13]:
#Linear linked messages
#get keys
(N, e), (p, q, d) = get_keys()
R.<x> = PolynomialRing(Zmod(N))

a1 = random.randint(2, N-2)
b1 = random.randint(2, N-1)
a2 = random.randint(2, N-1)
b2 = random.randint(2, N-1)
f1 = a1*x + b2
f2 = a2*x + b2

m = int(bytes_to_long(b'this_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_message')) 

c1 = pow(int(f1(m)), e, N)
c2 = pow(int(f2(m)), e, N)

In [16]:
m_decr = franklin_reiter(f1, f2, c1, c2, e, N)
m_decr = int(m_decr)

x + 21116652330039465980302856239322027310251817155196199544452626409535294311140590939570163149383212599236964200809202005450805701469083139918179697246401954978623125919181358309928632650358971722606970220709098735004380639383898484620714409219747511356848531606299565975288136622406184054996859805722375796322158883819836850185117473561772167249857813727725826314734562290614326294764396581720007564954121290554685681460688173243978421479656192683036631016875061198138953002811803252146943307178959751466404624589771902473529217866756401482175574666481501255378753985035478540357545646801008689185838486002482320873398


In [17]:
long_to_bytes(m_decr)

b'this_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_messagethis_is_a_secret_message'

# Resources

- https://en.wikipedia.org/wiki/Coppersmith%27s_attack
- https://eprint.iacr.org/2009/205.pdf
- https://www.researchgate.net/publication/221010549_A_new_related_message_attack_on_RSA/link/0deec534c1081b034d000000/download
- https://crypto.stackexchange.com/questions/30884/help-understanding-basic-franklin-reiter-related-message-attack
- https://translate.google.com/translate?hl=en&sl=zh-CN&u=https://ctf-wiki.github.io/ctf-wiki/crypto/asymmetric/rsa/rsa_coppersmith_attack-zh/&prev=search&pto=aue